# Automatic generation of image-captions

Today's tutorial aims at showing you the capabilities that neural networks have when dealing with images and language. We will see how to use some pre-trained neural networks to program our image-caption generator: essentially we will create a simple system which given any image as input, will output a brief description of the image. This system will consist of two neural networks, the first one will be responsible for dealing with the visual part of the problem (called the encoder), while the second one with the linguistic one (called the decoder). We will not train these neural networks ourselves, therefore you will need to download
the already trained weights yourself from the following links:

 * [weights](https://www.dropbox.com/s/ne0ixz5d58ccbbz/pretrained_model.zip?dl=0)
 
We will also need a list of words that the neural network will use for creating the captions of the images. 

 * [vocabulary](https://www.dropbox.com/s/26adb7y9m98uisa/vocap.zip?dl=0)
 
 ![alt text](images/model.png)

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from torchvision import transforms 
from build_vocab import Vocabulary
from model import EncoderCNN, DecoderRNN
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Step 1: load an image we want to generate a caption for

Write a custom function which given a path to an image loads it with the PIL library.
Explore the type of the object which gets loaded, do you think it is possible to use
such an object as input to a neural network? Why/Why not?

#### Step 2: convert the previously loaded image to a tensor

An Image object cannot be used as input for a neural network. We need in fact to convert it to a tensor and normalize the values of its pixels.

Write a custom function that gets a PIL Image as input, normalizes its pixels and converts the image to a tensor which can be used as input for a neural network.

#### Step 3: load the vocabulary of words which will be used for creating a caption

The neural network will require a pool of words to choose from to come up with an appropriate caption for an image. You should have already downloaded a pickle file consisting of a vocabulary which you just need to load in the coming function.

#### Step 4: create the computational graph for the neural architectures we will use

Just for today's tutorial we already provide you with such functions. The neural network consists of two different parts, one denoted as an encoder and one as a decoder, we, therefore, need two different functions. 

In [8]:
def build_encoder(embedding_size):
    encoder = EncoderCNN(embedding_size).eval()  # eval mode (batchnorm uses moving mean/variance)
    encoder = encoder.to(device)
    
    return encoder

In [9]:
def build_decoder(embedding_size, hidden_size, vocab, num_layers):
    decoder = DecoderRNN(embedding_size, hidden_size, len(vocab), num_layers)
    decoder = decoder.to(device)
    return decoder

#### Step 5: load appropriate weights within the computational graph.

The networks which have been created with the previous two functions are not trained, meaning that the captions which we would obtain as output will be very likely garbage. To get better results we need to load the appropriate weights within the computation graph. Write a function which given a computation graph loads the already trained weights that you have downloaded.

#### Step 6: extract meaningful information from the encoder.

A tensor representing an image does not give us much information. We aim at converting such an image to a feature vector that represents the original image in a meaningful way. This corresponds to computing the *forward pass* in the network. Write a function that gets as input an image, the encoder-network and returns a feature vector.

What is the difference between the extracted features and the original input of the network?

#### Step 7: match the extracted features with a potential caption

We now aim at using the features which have been extracted by the encoder-network and use them as input for the decoder-network. The goal is to get from the decoder a set of words which will consists in a potential caption for the image. Write a function which gets the decoder-network and the previously extracted features as input and returns what will later be the words of the caption. 

**TIP**: the decoder will call the .sample() function defined in model.py, so take a look at that python file.

#### Step 8: convert the output of the decoder to meaningful words

The decoder network does not output directly words but a list of numbers instead.
We now convert these numbers to meaningful words with the following function.

In [73]:
def create_caption(sampled_ids):
    sampled_caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            return sampled_caption

In [10]:
def main():
    
    # parameters to pass to build_encoder() and build_decoder()
    embedding_size = 256
    hidden_size = 512
    num_layers = 1 
    
main()